# Import libraries

In [36]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
from time import sleep
from random import randint

# Information I want to request

In [37]:
#Data information

job_title= []
company= []
location= []
post_date = []
today=[]
summary=[]
job_url=[]
salary=[]


# Create a function for extracting data

In [38]:

def extract_jobs():
    """This functions works for extracting job data"""
    pages = np.arange(0, 100, 10)
    #Navigate through pages
    for page in pages: 
        try:
            #Create soup
            page=requests.get("https://mx.indeed.com/jobs?q=data+scientist&start="+str(page))
            soup=BeautifulSoup(page.text,'html.parser')
            jobs=soup.find_all('div','jobsearch-SerpJobCard')
            sleep(randint(2,5))
            #Adding data
            for job in jobs:
                
                #Extract title
                atag=job.h2.a
                title=atag.get('title')
                job_title.append(title)
                
                #Extract company name
                companies=job.find('span','company').text.strip()
                company.append(companies)
                
                #Extract location
                location_job=job.find('div','recJobLoc').get('data-rc-loc')
                location.append(location_job)
                
                #Extract date of posting
                post=job.find('span','date').text.replace(u"\xa0",u" ").replace("hace ","").replace(" días","").replace("Hace más de 30","+30").replace("Hoy","0")
                post_date.append(post)
                
                #Scraping date
                today_date=datetime.today().strftime('%Y-%m-%d')
                today.append(today_date)

                
                #Extract summary info
                summary_list=job.find('div','summary').text.strip().replace('\n',' ')        
                summary.append(summary_list)

                #Extract job URL's
                job_urls='https://mx.indeed.com'+atag.get('href')
                job_url.append(job_urls)
            
                #Extract salary
                try:
                    salaries=job.find('span','salaryText').text.strip()
                    salary.append(salaries)
                except AttributeError:
                    salaries="None"
                    salary.append(salaries)

        except AttributeError:
            break


extract_jobs()


In [39]:
#Create a DataFrame and cleaning last columns
list_of_jobs=pd.DataFrame({
    'Job_title':job_title,
    'Company':company,
    'Location':location,
    'Salary':salary,
    'Date_of_post':post_date,
    'Today_date':today,
    'Summary':summary,
    'Job_URL':job_url
})





In [40]:
list_of_jobs

,Job_title,Company,Location,Salary,Date_of_post,Today_date,Summary,Job_URL
0,Data Scientist/Machine Learning Engineer,Consultoria en TI,Desde casa,"$55,000 - $62,000 al mes",10,2020-11-22,Experienced with data wrangling (data cleaning...,https://mx.indeed.com/company/Consultoria-en-T...
1,Data Scientist,BEST PRACTICES CONSULTING SA DE CV,"Querétaro, Qro.","$25,000 al mes",19,2020-11-22,"Diseño de arquitecturas, planes de implementac...",https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...
2,Data Scientist,Toshiba Global Commerce Solutions,"Colonia Lomas de Chapultepec, CDMX","$30,000 - $40,000 al mes",18,2020-11-22,Defines company data assets (data models) to p...,https://mx.indeed.com/company/Toshiba-Global-C...
3,Data Scientist,Unilever,"Colonia Bosques de las Lomas, CDMX",None,+30,2020-11-22,Experiencia mínima de 1 año como Data Scientis...,https://mx.indeed.com/rc/clk?jk=01cd43eb2b84de...
4,Data Scientist (Experiencia en Mkt. Digital),Kontrabando,"Colonia Polanco, CDMX",None,3,2020-11-22,Ability to easily articulate the value proposi...,https://mx.indeed.com/rc/clk?jk=8daa285b2cdfe1...
...,...,...,...,...,...,...,...,...
132,Senior Site Reliability Engineer,Urbint,Ciudad de México,None,+30,2020-11-22,"We are a tight-knit team of coders, data scien...",https://mx.indeed.com/rc/clk?jk=a1996624f8f312...
133,Senior Engineer,Aspen Technology,Ciudad de México,None,24,2020-11-22,The driving force behind our success has alway...,https://mx.indeed.com/rc/clk?jk=f5be61b6ff7ed0...
134,Senior Data Engineer,StartupGDL,"Guadalajara, Jal.",None,+30,2020-11-22,"You will work with data scientists, product ma...",https://mx.indeed.com/rc/clk?jk=575c7c14bf03da...
135,Senior Software Developer,Aspen Technology,Ciudad de México,None,24,2020-11-22,The driving force behind our success has alway...,https://mx.indeed.com/rc/clk?jk=e0dcf82ec9b7eb...


In [12]:
list_of_jobs.to_csv("Jobs_list.csv")

# This will concat information obtained and let us do some research

In [41]:
#We can use the function to search other jobs and obtain a data set
data_science=pd.read_csv("Jobs_list.csv")
data_engineer=pd.read_csv("Jobs_list_2.csv")
data_jobs=pd.concat([data_science,data_engineer],axis=0).drop_duplicates()
data_jobs.shape
#print(data_jobs.isnull().sum())


Unnamed: 0      0
Job_title       0
Company         0
Location        0
Salary          0
Date_of_post    0
Today_date      0
Summary         0
Job_URL         0
dtype: int64


In [45]:
#Companies with the highest postings
comp_count=data_jobs.groupby("Company").count()["Job_title"].sort_values(ascending=False)[:10]
comp_count

Company
Microsoft                 11
Aspen Technology           9
Kueski                     8
John Deere                 7
Flex                       6
Jabil                      5
Stanley Black & Decker     5
Credijusto                 4
Bosch Group                4
Oyster Financial Inc       4
Name: Job_title, dtype: int64

In [46]:
#Location with the highest job listing
loc_count=data_jobs.groupby("Location").count()["Job_title"].sort_values(ascending=False)[:10]
loc_count

Location
Ciudad de México         69
Guadalajara, Jal.        26
Querétaro, Qro.          18
Monterrey, N. L.         13
Desde casa               12
Mexicali, B.C.            9
Tijuana, B.C.             8
Colonia Polanco, CDMX     6
Miguel Hidalgo, CDMX      6
Álvaro Obregón, CDMX      4
Name: Job_title, dtype: int64